<a href="https://colab.research.google.com/github/handaru/data/blob/main/colab/data_engineering/Advanced_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [ ]:
%load_ext google.colab.data_table

In [ ]:
project_id = 'trusty-shine-430910-t0'

from google.cloud import bigquery

client = bigquery.Client(project=project_id)

In [ ]:
df = client.query('''
  SELECT
  `hash` AS transaction_hash,
  block_hash
FROM
  `bigquery-public-data.crypto_ethereum.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
  AND block_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND parent_hash = "0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad90bcd0623c8b7f5b6efe")
LIMIT
  1000''').to_dataframe()

df.head()

,transaction_hash,block_hash
0,0x19d5079668746ba88d7c990ed33116c17e79ceb45bd4...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
1,0x7d80687bbd216dec26b1a33fb8bd30941b5d6a9c034a...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
2,0x165969506456847a907dfaece58270afea5b149c0d93...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
3,0xe9361386998ce6117271c1621a848e31a628aeb2019a...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...
4,0x317174ddb7034d7544a45b056543b7a1632159140760...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...


In [ ]:
df = client.query('''
  SELECT
  t.transaction_hash,
  t.block_hash,
  tb.parent_hash
FROM (
  SELECT
    `hash` AS transaction_hash,
    block_hash
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")) AS t
INNER JOIN (
  SELECT
    `hash` AS block_hash,
    parent_hash
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND parent_hash = "0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad90bcd0623c8b7f5b6efe" ) AS tb
ON
  t.block_hash = tb.block_hash
ORDER BY
  t.transaction_hash
LIMIT
  1000''').to_dataframe()

df.head()

,transaction_hash,block_hash,parent_hash
0,0x00f2d3227b470e4d31d2066bd4215fed669a21e5a570...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
1,0x01606a9bb282c68fa477662c2b6ccca758b9a1782448...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
2,0x018df5d15cf63ba4fc980b20ba54a023b4c2b24c67aa...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
3,0x0420d46f8d2027d43b3fce1fb4d34884bfdf226596e0...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
4,0x047800c856069dc51b0e90230884a79660e415b1cb33...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...


In [ ]:
df = client.query('''
  (SELECT
  geo_id
FROM
  `bigquery-public-data.census_bureau_usa.population_by_zip_2000`
LIMIT
  10)
UNION DISTINCT
(SELECT
  geo_id
FROM
  `bigquery-public-data.census_bureau_usa.population_by_zip_2010`
LIMIT
  10)''').to_dataframe()

df.head()

,geo_id
0,8600000US25647
1,8600000US49336
2,8600000US72106
3,8600000US74074
4,8600000US01960


In [ ]:
df = client.query('''
  SELECT
  block_hash,
  COUNT(`hash`) AS total_transaction
FROM
  `bigquery-public-data.crypto_ethereum.transactions`
WHERE
  TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
  AND block_hash IN (
  SELECT
    `hash`
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
GROUP BY
  (block_hash)''').to_dataframe()

df.head()

,block_hash,total_transaction
0,0xb7ced46bf0a054f043b992235ec19e7f40951ce2ad6c...,208
1,0x5bd588de26eb95a8945325beb9cb45239b79d3ef0d60...,197
2,0x1dfed1918d52c71a5902a5e1f5fb9c5dcb0b45b9e2ff...,272
3,0xe6e7005294fa650fa9b14e5b305815a150788ae30ae4...,184
4,0x523ef0127ffcccc13385d400e42b7cbec32214bdd67c...,256


from matplotlib import pyplot as plt
_df_0['total_transaction'].plot(kind='hist', bins=20, title='total_transaction')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
_df_1.groupby('block_hash').size().plot(kind='barh', color=sns.palettes.mpl_palette('Dark2'))
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['total_transaction'].plot(kind='line', figsize=(8, 4), title='total_transaction')
plt.gca().spines[['top', 'right']].set_visible(False)

<string>:5: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.



from matplotlib import pyplot as plt
import seaborn as sns
figsize = (12, 1.2 * len(_df_3['block_hash'].unique()))
plt.figure(figsize=figsize)
sns.violinplot(_df_3, x='total_transaction', y='block_hash', inner='stick', palette='Dark2')
sns.despine(top=True, right=True, bottom=True, left=True)

In [ ]:
df = client.query('''
SELECT
  block_hash,
  total_transaction,
  ROW_NUMBER() OVER (ORDER BY total_transaction DESC) AS rank
FROM (
  SELECT
    block_hash,
    COUNT(`hash`) AS total_transaction
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND block_hash IN (
    SELECT
      `hash`
    FROM
      `bigquery-public-data.crypto_ethereum.blocks`
    WHERE
      TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06"))
  GROUP BY
    (block_hash))''').to_dataframe()

df.head()

,block_hash,total_transaction,rank
0,0xd21124debd7f04c43d8c2697b0f4995664ebd69ebabd...,894,1
1,0xad1bb588af8d932a91e2b4b875dccaff641c0590c286...,777,2
2,0xfab03139f33dff846394e076e6e0d4f4c1c4cf2f7984...,767,3
3,0xb18712c4f93c81186f1cda73ee52e577aabaa2b51a63...,763,4
4,0x02e720d3d306f3eacb0ebbae3ea589e1b4e32e64937b...,733,5


In [ ]:
df = client.query('''
WITH
  transaction_block AS (
  SELECT
    `hash` AS transaction_hash,
    block_hash
  FROM
    `bigquery-public-data.crypto_ethereum.transactions`
  WHERE
    TIMESTAMP_TRUNC(block_timestamp, DAY) = TIMESTAMP("2024-08-06")),
  block_parent AS (
  SELECT
    `hash` AS block_hash,
    parent_hash
  FROM
    `bigquery-public-data.crypto_ethereum.blocks`
  WHERE
    TIMESTAMP_TRUNC(timestamp, DAY) = TIMESTAMP("2024-08-06")
    AND parent_hash = "0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad90bcd0623c8b7f5b6efe" )
SELECT
  transaction_block.transaction_hash,
  transaction_block.block_hash,
  block_parent.parent_hash
FROM
  transaction_block
INNER JOIN
  block_parent
ON
  transaction_block.block_hash = block_parent.block_hash
ORDER BY
  transaction_block.transaction_hash
LIMIT
  1000''').to_dataframe()

df.head()

,transaction_hash,block_hash,parent_hash
0,0x00f2d3227b470e4d31d2066bd4215fed669a21e5a570...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
1,0x01606a9bb282c68fa477662c2b6ccca758b9a1782448...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
2,0x018df5d15cf63ba4fc980b20ba54a023b4c2b24c67aa...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
3,0x0420d46f8d2027d43b3fce1fb4d34884bfdf226596e0...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
4,0x047800c856069dc51b0e90230884a79660e415b1cb33...,0xd4e61fbfa7ea696beb85d1c98a738db0a22ec8a95223...,0x1a0bd10d0603122a3e1bfb95b91f7bfcbb7be54953ad...
